In [ ]:
# For package management
workdir_ = '/glade/work/juliob/'
import sys

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Get some regirdding stuff
sys.path.append(workdir_ + 'PyRegridding/Regridder/')
import scripGen as SG
import esmfmeshGen as emG

# Some other useful packages 
import importlib
import esmpy as E

# reload dev codes
importlib.reload( SG )
importlib.reload( emG )


In [ ]:
from scipy.ndimage import uniform_filter1d
import copy

def smooth_sst_ndarray_with_boxcar(sst_array , ocn_msk, iterate=1, size=[3,3] ):
    """
    Smooths the SST field using a boxcar filter, replacing land (fill_value) with smoothed values.
    
    Parameters:
    - sst_array: 3D numpy ndarray containing SST data with dimensions (nt, ny, nx)
    - fill_value: Value representing land (default is -10)
    - size: Size of the boxcar filter window (default is 3x3)

    Returns:
    - sst_smoothed: 3D ndarray with land values replaced by smoothed SST values.
    """
    
    # Step 2: Initialize an array to hold the smoothed results
    sst_smoothed = copy.deepcopy(sst_array) #np.empty_like(sst_array)

    for nit in np.arange( iterate):
        # Loop over time dimension and apply the boxcar filters to each 2D slice
        for t in range(sst_array.shape[0]):  # Loop over the time dimension (nt)
    
            # Step 1: Apply boxcar smoothing along the latitude axis (ny)
            sst_lat_smoothed = uniform_filter1d(sst_smoothed[t], size=size[0], axis=0, mode='reflect')
            
            # Step 2: Apply boxcar smoothing along the longitude axis (nx) with a different boundary treatment
            sst_smoothed[t] = uniform_filter1d(sst_lat_smoothed, size=size[1], axis=1, mode='wrap' ) #, cval=cval_lon)
        
    
        sst_smoothed = np.where( ocn_msk == 1, sst_array , sst_smoothed )
        print( f"Iteration={nit}" )

    return sst_smoothed
    
from scipy.ndimage import uniform_filter

def smooth_sst_ndarray_with_boxcar_fast(sst_array, ocn_msk, iterate=1, size=(3,3)):
    """
    Smooths the SST field using a boxcar filter, replacing land with smoothed values.
    
    Parameters:
    - sst_array: 3D numpy ndarray containing SST data with dimensions (nt, ny, nx)
    - ocn_msk: 2D numpy array representing the ocean mask (1 for ocean, 0 for land)
    - iterate: Number of iterations for smoothing
    - size: Size of the boxcar filter window (default is 3x3)

    Returns:
    - sst_smoothed: 3D ndarray with land values replaced by smoothed SST values.
    """
    # Initialize a copy for smoothing
    #sst_smoothed = np.copy(sst_array)
    sst_smoothed = copy.deepcopy(sst_array) #np.empty_like(sst_array)

    for nit in range(iterate):
        # Apply 2D boxcar filter to each time slice (nt, ny, nx)
        sst_smoothed = uniform_filter(sst_smoothed, size=(1, *size), mode=('reflect', 'reflect', 'wrap'))
        
        # Replace land points (ocn_msk == 1) with original SST values
        sst_smoothed = np.where(ocn_msk == 1, sst_array, sst_smoothed)
        print(f"Iteration={nit}")
    
    return sst_smoothed

import cftime
import datetime

def generate_no_leap_dates(start_date, end_date):
    """
    Generate a list of cftime.DatetimeNoLeap dates for a no-leap calendar, from start_date to end_date.
    """
    dates = []
    current_date = start_date
    delta = datetime.timedelta(days=1)  # Use the standard datetime timedelta

    while current_date <= end_date:
        dates.append(current_date)
        # Move to the next day (using datetime timedelta)
        current_date = current_date + delta
    
    return dates



In [ ]:
#######################################
# Make time, lat, lon coordinates and 
# time variables
#######################################

# Step 1: Define the start and end date
start_date = cftime.DatetimeNoLeap(2003, 1, 1, 12, 0, 0, has_year_zero=True)
end_date = cftime.DatetimeNoLeap(2005, 12, 31, 12, 0, 0, has_year_zero=True)

# Step 2: Generate daily dates using cftime
cftime_dates = generate_no_leap_dates(start_date, end_date)

# Step 3: Create an xarray DataArray with the generated time dimension
time_da = xr.DataArray(cftime_dates, dims='time', name='time')

# Example usage in a Dataset
# Define the latitude and longitude arrays
nx , ny = 1440, 720

latitudes = np.linspace(-90, 90, ny + 1)  # ny+1 grid points in latitude
longitudes = np.linspace(0, 360, nx, endpoint=False)  # nx grid points in longitude, periodic in lon
delta_lon = (360.) / nx  # Longitude cell width in radians
center_lons = longitudes + 0.5*delta_lon 
center_lats = 0.5*( latitudes[1:]+latitudes[0:-1] )


ds = xr.Dataset(
    coords={
        'time': time_da,
        'lat': center_lats,
        'lon': center_lons
    }
)

#######################
print(len( ds.time ) )

ntimes=len( ds.time ) 
date = np.zeros( ntimes, dtype=np.int32 )
datesec = np.zeros( ntimes, dtype=np.int32 )

for n in np.arange( ntimes ):
    date[n] = 10_000*ds.time[n].item().year+100*ds.time[n].item().month+ds.time[n].item().day
    datesec[n] = 3_600*ds.time[n].item().hour

ds['date'] = xr.DataArray( date.astype(np.int32) ,
                             dims=('time',)) 
ds['datesec'] = xr.DataArray( datesec.astype(np.int32) ,
                             dims=('time',)) 


In [ ]:
ds

In [ ]:
####
#   
####
files=['/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/sst.day.mean.2003.nc', '/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/sst.day.mean.2004.nc', '/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/sst.day.mean.2005.nc' ]

SST=xr.open_mfdataset(files, concat_dim='time', combine='nested' )
# Remove February 29, 2004
SST = SST.sel(time=~((SST['time.month'] == 2) & (SST['time.day'] == 29) & (SST['time.year'] == 2004)))

sstnan=SST.sst.values

SST_zer=SST.fillna( 0.)

sstv=SST_zer.sst.values
nt,ny,nx=np.shape( sstv )

sstZon=np.nanmean( sstnan, axis=2 )
sstZonX=np.expand_dims( sstZon, axis=2)
sstZonX= np.repeat( sstZonX, nx , axis=2)


sstFill = np.where(np.isnan(sstnan), sstZonX , sstv)
sstFill = np.where(np.isnan(sstFill), 0. , sstFill)

ocn_msk = np.where(np.isnan(sstnan), 0. , 1. )


sstFillSm = smooth_sst_ndarray_with_boxcar(sstFill , ocn_msk, iterate=10, size=[17,17] )

In [ ]:
plt.contourf( (sstFillSm)[360,:,:], levels=np.linspace(-10,40,num=51) )

In [ ]:
#   
####
files=['/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/icec.day.mean.2003.nc', '/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/icec.day.mean.2004.nc', '/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/icec.day.mean.2005.nc' ]

ICE=xr.open_mfdataset(files, concat_dim='time', combine='nested' )
# Remove February 29, 2004
ICE = ICE.sel(time=~((ICE['time.month'] == 2) & (ICE['time.day'] == 29) & (ICE['time.year'] == 2004)))

icenan=ICE.icec.values

ICE_zer=ICE.fillna( 0.)
icev=ICE_zer.icec.values

In [ ]:
################################
# Add SST and ice_cover
################################

ds['SST_cpl'] = xr.DataArray( sstFillSm.astype(np.float32) ,
                             dims=('time','lat','lon',)) 
ds['ice_cov'] = xr.DataArray( icev.astype(np.float32) ,
                             dims=('time','lat','lon',)) 


ds.to_netcdf( '/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/sst_ice_NOAA_QxQ_2003_2005.nc' )


In [ ]:
Mesh=emG.nxny_to_mesh( nx=nx, ny=ny, mesh_file_out='test-mesh.nc' )

In [ ]:
Mesh.to_netcdf( '/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/sst_ice_NOAA_QxQ_ESMFmesh.nc' )